In [ ]:
# The cells in this notebook provide the R or SAS sample code provided
# by the BLS. https://www.bls.gov/cex/pumd-getting-started-guide.htm#section5


In [ ]:
THIS RECREATES TABLE 1203 INCOME BEFORE TAXES 

  /***************************************************************************/
  /* PROGRAM NAME:  CEX INTEGRATED SURVEYS SAMPLE PROGRAM (SAS)              */
  /* FUNCTION: CREATE AN INTEGRATED SURVEY EXPENDITURE TABLE BY INCOME CLASS */
  /*           USING MICRODATA FROM THE BUREAU OF LABOR STATISTICS' CONSUMER */
  /*           EXPENDITURE SURVEY.                                           */
  /*                                                                         */
  /* WRITTEN BY: BUREAU OF LABOR STATISTICS         APRIL 7 2003             */
  /*             CONSUMER EXPENDITURE SURVEY                                 */
  /* MODIFICATIONS:                                                          */
  /* DATE-      MODIFIED BY-        REASON-                                  */
  /* -----      ------------        -------                                  */
  /*                                                                         */ 
  /*                                                                         */
  /*                                                                         */
  /*  NOTE:  FOR SAS VERSION 8 OR HIGHER                                     */
  /***************************************************************************/


  /*Enter Data Year*/
    %LET YEAR = 2018;
  /*Enter location of the unzipped microdata file*/
  /*Be sure to keep the same file structure as found online*/
    %LET DRIVE = C:\2018_CEX;


  /***************************************************************************/
  /* STEP1: READ IN THE STUB PARAMETER FILE AND CREATE FORMATS               */
  /* ----------------------------------------------------------------------- */
  /* 1 CONVERTS THE STUB PARAMETER FILE INTO A LABEL FILE FOR OUTPUT         */
  /* 2 CONVERTS THE STUB PARAMETER FILE INTO AN EXPENDITURE AGGREGATION FILE */
  /* 3 CREATES FORMATS FOR USE IN OTHER PROCEDURES                           */
  /***************************************************************************/


%LET YR1 = %SUBSTR(&YEAR, 3, 2);
%LET YR2 = %SUBSTR(%EVAL(&YEAR + 1), 3, 2);
LIBNAME I&YR1 "&DRIVE\INTRVW&YR1";
LIBNAME D&YR1 "&DRIVE\DIARY&YR1";

DATA STUBFILE (KEEP= COUNT TYPE LEVEL TITLE UCC SURVEY GROUP LINE);
  INFILE "C:\2018_CEX\Programs\INTSTUB&YEAR..TXT"
  PAD MISSOVER;
  INPUT @1 TYPE $1. @ 4 LEVEL $1. @7 TITLE $CHAR60. @70 UCC $6.
        @83 SURVEY $1. @89 GROUP $7.;
  IF (TYPE = '1');
  IF GROUP IN ('CUCHARS','FOOD','EXPEND','INCOME');
  
    RETAIN COUNT 9999;
    COUNT + 1;
    LINE = PUT(COUNT, $5.)||LEVEL;
	/* READS IN THE STUB PARAMETER FILE AND CREATES LINE NUMBERS FOR UCCS */
	/* A UNIQUE LINE NUMBER IS ASSIGNED TO EACH EXPENDITURE LINE ITEM     */
RUN;


DATA AGGFMT1 (KEEP= UCC LINE LINE1-LINE10);
  SET STUBFILE;
  LENGTH LINE1-LINE10 $6.;
    ARRAY LINES(9) LINE1-LINE9;
      IF (UCC > 'A') THEN
        LINES(SUBSTR(LINE,6,1)) = LINE;
	  RETAIN LINE1-LINE9;
      IF (UCC < 'A')  THEN 
        LINE10 = LINE;
  IF (LINE10);
  /* MAPS LINE NUMBERS TO UCCS */
RUN;


PROC SORT DATA= AGGFMT1 (RENAME=(LINE= COMPARE));
  BY UCC;
RUN;


PROC TRANSPOSE DATA= AGGFMT1 OUT= AGGFMT2 (RENAME=(COL1= LINE));
  BY UCC COMPARE;
  VAR LINE1-LINE10;
RUN;


DATA AGGFMT (KEEP= UCC LINE);
  SET AGGFMT2;
    IF LINE;
    IF SUBSTR(COMPARE,6,1) > SUBSTR(LINE,6,1) OR COMPARE=LINE;
	/* AGGREGATION FILE. EXTRANEOUS MAPPINGS ARE DELETED */
	/* PROC SQL WILL AGGANGE LINE#/UCC PAIRS FOR USE IN PROC FORMAT */
	
RUN;


PROC SQL NOPRINT;
  SELECT UCC, LINE, COUNT(*)
  INTO  :UCCS SEPARATED BY " ",
        :LINES SEPARATED BY " ",          
        :CNT
  FROM AGGFMT;
  QUIT;
RUN;


%MACRO MAPPING;
  %DO  i = 1  %TO  &CNT;
    "%SCAN(&UCCS,&i,%STR( ))" = "%SCAN(&LINES,&i,%STR( ))"
  %END;
%MEND MAPPING;


DATA LBLFMT (RENAME=(LINE= START TITLE= LABEL));
  SET STUBFILE (KEEP= LINE TITLE);
  RETAIN FMTNAME 'LBLFMT' TYPE 'C';
  /* LABEL FILE. LINE NUMBERS ARE ASSIGNED A TEXT LABEL */
  /* DATASET CONSTRUCTED TO BE READ INTO A PROC FORMAT  */
RUN;


PROC FORMAT;

  VALUE $AGGFMT (MULTILABEL)
    %MAPPING
    OTHER= 'OTHER'
    ;

  VALUE $INC (MULTILABEL)
    '01' = '01'
    '01' = '10'
    '02' = '02'
    '02' = '10'
    '03' = '03'
    '03' = '10'
    '04' = '04'
    '04' = '10'
    '05' = '05'
    '05' = '10'
    '06' = '06'
    '06' = '10'
    '07' = '07'
    '07' = '10'
    '08' = '08'
    '08' = '10'
    '09' = '09'
    '09' = '10';
	/* CREATE INCOME CLASS FORMAT */
RUN;


PROC FORMAT LIBRARY= WORK  CNTLIN= LBLFMT;
RUN;


  /***************************************************************************/
  /* STEP2: READ IN ALL NEEDED DATA                                          */
  /* ----------------------------------------------------------------------- */
  /* 1 READ IN THE INTERVIEW AND DIARY FMLY FILES & CREATE MO_SCOPE VARIABLE */
  /* 2 READ IN THE INTERVIEW MTAB/ITAB AND DIARY EXPN/DTAB FILES             */
  /* 3 MERGE FMLY AND EXPENDITURE FILES TO DERIVE WEIGHTED EXPENDITURES      */
  /***************************************************************************/


DATA FMLY (KEEP= NEWID SOURCE INCLASS WTREP01-WTREP44 FINLWT21 REPWT1-REPWT45);

SET D&YR1..FMLD&YR1.1
    D&YR1..FMLD&YR1.2
    D&YR1..FMLD&YR1.3
    D&YR1..FMLD&YR1.4

    I&YR1..FMLI&YR1.1x (IN=FIRSTQTR)
    I&YR1..FMLI&YR1.2 
    I&YR1..FMLI&YR1.3
    I&YR1..FMLI&YR1.4
    I&YR1..FMLI&YR2.1  (IN= LASTQTR);

	BY NEWID;

    IF FIRSTQTR THEN 
      MO_SCOPE = (QINTRVMO - 1);
    ELSE IF LASTQTR THEN 
      MO_SCOPE = (4 - QINTRVMO);
    ELSE  
      MO_SCOPE = 3;

	
    ARRAY REPS_A(45) WTREP01-WTREP44 FINLWT21;
    ARRAY REPS_B(45) REPWT1-REPWT45;

      DO i = 1 TO 45;
	  IF REPS_A(i) > 0 THEN
         REPS_B(i) = (REPS_A(i) * MO_SCOPE / 12); 
		 ELSE REPS_B(i) = 0;	
	  END;

	  IF QINTRVYR  THEN 
        SOURCE = 'I';
	  IF WEEKI THEN 
        SOURCE = 'D';

/******CREATE VARIABLE INCLASS FOR INTERVIEW SURVEY*********/
	  IF FINCBTXM NE . AND FINCBTXM < 15000 THEN INCLASS = '01';
	  IF 15000 <=  FINCBTXM < 30000 THEN INCLASS = '02';
	  IF 30000 <=  FINCBTXM < 40000 THEN INCLASS = '03';
	  IF 40000 <=  FINCBTXM < 50000 THEN INCLASS = '04';
	  IF 50000 <=  FINCBTXM < 70000 THEN INCLASS = '05';
	  IF 70000 <=  FINCBTXM < 100000 THEN INCLASS = '06';
	  IF 100000 <= FINCBTXM < 150000 THEN INCLASS = '07';
	  IF 150000 <= FINCBTXM < 200000 THEN INCLASS = '08';
	  IF FINCBTXM NE . AND FINCBTXM >= 200000  THEN INCLASS = '09';
/******CREATE VARIABLE INCLASS FOR DIARY SURVEY*********/
	  IF FINCBEFM NE . AND FINCBEFM < 15000 THEN INCLASS = '01';
	  IF 15000 <=  FINCBEFM < 30000 THEN INCLASS = '02';
	  IF 30000 <=  FINCBEFM < 40000 THEN INCLASS = '03';
	  IF 40000 <=  FINCBEFM < 50000 THEN INCLASS = '04';
	  IF 50000 <=  FINCBEFM < 70000 THEN INCLASS = '05';
	  IF 70000 <=  FINCBEFM < 100000 THEN INCLASS = '06';
	  IF 100000 <= FINCBEFM < 150000 THEN INCLASS = '07';
	  IF 150000 <= FINCBEFM < 200000 THEN INCLASS = '08';
	  IF FINCBEFM NE . AND FINCBEFM >= 200000  THEN INCLASS = '09';
	   
RUN;

DATA EXPEND (KEEP= NEWID SOURCE UCC COST REFMO REFYR);

  SET D&YR1..EXPD&YR1.1
      D&YR1..EXPD&YR1.2
      D&YR1..EXPD&YR1.3
      D&YR1..EXPD&YR1.4

      D&YR1..DTBD&YR1.1 (RENAME=(AMOUNT=COST))
      D&YR1..DTBD&YR1.2 (RENAME=(AMOUNT=COST))
      D&YR1..DTBD&YR1.3 (RENAME=(AMOUNT=COST))
      D&YR1..DTBD&YR1.4 (RENAME=(AMOUNT=COST))

      I&YR1..MTBI&YR1.1X
      I&YR1..MTBI&YR1.2
      I&YR1..MTBI&YR1.3
      I&YR1..MTBI&YR1.4
      I&YR1..MTBI&YR2.1

      I&YR1..ITBI&YR1.1X  (RENAME=(VALUE=COST))
      I&YR1..ITBI&YR1.2   (RENAME=(VALUE=COST))
      I&YR1..ITBI&YR1.3   (RENAME=(VALUE=COST))
      I&YR1..ITBI&YR1.4   (RENAME=(VALUE=COST))
      I&YR1..ITBI&YR2.1   (RENAME=(VALUE=COST)); 
  
  IF (PUBFLAG = '2') THEN 
    DO;
		SOURCE = 'I';
        IF (REFYR = "&YEAR") OR  (REF_YR = "&YEAR") THEN 
        OUTPUT;
    END;

  IF (PUB_FLAG = '2') THEN 
    DO;
      SOURCE = 'D';
	  COST = (COST * 13);
      OUTPUT;
    END;
RUN;

PROC SORT DATA=EXPEND;
	BY NEWID;
RUN;

DATA PUBFILE (KEEP= NEWID SOURCE INCLASS UCC RCOST1-RCOST45);
  MERGE FMLY   (IN= INFAM)
        EXPEND (IN= INEXP);
  BY NEWID;
  IF (INEXP AND INFAM);

  IF (COST = .)  THEN 
     COST = 0;
	 
     ARRAY REPS_A(45) WTREP01-WTREP44 FINLWT21;
     ARRAY REPS_B(45) RCOST1-RCOST45;

     DO i = 1 TO 45;
	   IF REPS_A(i) > 0
         THEN REPS_B(i) = (REPS_A(i) * COST);
	     ELSE REPS_B(i) = 0; 	
	 END; 
RUN;



  /***************************************************************************/
  /* STEP3: CALCULATE POPULATIONS                                            */
  /* ----------------------------------------------------------------------- */
  /*  SUM ALL 45 WEIGHT VARIABLES TO DERIVE REPLICATE POPULATIONS            */
  /*  FORMATS FOR CORRECT COLUMN CLASSIFICATIONS                             */
  /***************************************************************************/


PROC SUMMARY NWAY DATA=FMLY SUMSIZE=MAX;
  CLASS INCLASS SOURCE / MLF;
  VAR REPWT1-REPWT45;
  FORMAT INCLASS $INC.;
  OUTPUT OUT = POP (DROP = _TYPE_ _FREQ_) SUM = RPOP1-RPOP45;
RUN;

 

  /***************************************************************************/
  /* STEP4: CALCULATE WEIGHTED AGGREGATE EXPENDITURES                        */
  /* ----------------------------------------------------------------------- */
  /*  SUM THE 45 REPLICATE WEIGHTED EXPENDITURES TO DERIVE AGGREGATES/UCC    */
  /*  FORMATS FOR CORRECT COLUMN CLASSIFICATIONS                             */
  /***************************************************************************/


PROC SUMMARY NWAY DATA=PUBFILE SUMSIZE=MAX COMPLETETYPES;
  CLASS SOURCE UCC INCLASS / MLF;
  VAR RCOST1-RCOST45;
  FORMAT INCLASS $INC.;
   OUTPUT OUT= AGG (DROP= _TYPE_ _FREQ_) 
   SUM= RCOST1-RCOST45;
RUN;



  /***************************************************************************/
  /* STEP5: CALCULTATE MEAN EXPENDITURES                                     */
  /* ----------------------------------------------------------------------- */
  /* 1 READ IN POPULATIONS AND LOAD INTO MEMORY USING A 3 DIMENSIONAL ARRAY  */
  /*   POPULATIONS ARE ASSOCIATED BY INCLASS, SOURCE(t), AND REPLICATE(j)    */
  /* 2 READ IN AGGREGATE EXPENDITURES FROM AGG DATASET                       */
  /* 3 CALCULATE MEANS BY DIVIDING AGGREGATES BY CORRECT SOURCE POPULATIONS  */
  /*   EXPENDITURES SOURCED FROM DIARY ARE CALULATED USING DIARY POPULATIONS */
  /*   WHILE INTRVIEW EXPENDITURES USE INTERVIEW POPULATIONS                 */
  /* 4 SUM EXPENDITURE MEANS PER UCC INTO CORRECT LINE ITEM AGGREGATIONS     */
  /***************************************************************************/


DATA AVGS1 (KEEP = SOURCE INCLASS UCC MEAN1-MEAN45);

  /* READS IN POP DATASET. _TEMPORARY_ LOADS POPULATIONS INTO SYSTEM MEMORY  */
  ARRAY POP{01:10,2,45} _TEMPORARY_ ;
  IF _N_ = 1 THEN DO i = 1 TO 20;
    SET POP;
	ARRAY REPS{45} RPOP1--RPOP45;
	IF SOURCE = 'D' THEN t = 1;
	ELSE t = 2;
	  DO j = 1 TO 45;
	    POP{INCLASS,t,j} = REPS{j};
	  END;
	END;

  /* READS IN AGG DATASET AND CALCULATES MEANS BY DIVIDING BY POPULATIONS  */
  SET AGG (KEEP = UCC INCLASS SOURCE RCOST1-RCOST45);
	IF SOURCE = 'D' THEN t = 1;
	ELSE t = 2;
  ARRAY AGGS(45) RCOST1-RCOST45;
  ARRAY AVGS(45) MEAN1-MEAN45;
	DO k = 1 TO 45;
	  IF AGGS(k) = .  THEN AGGS(k) = 0;
	  AVGS(k) = AGGS(k) / POP{INCLASS,t,k};
	END;
RUN;


PROC SUMMARY DATA=AVGS1 NWAY COMPLETETYPES;
  CLASS INCLASS UCC / MLF;
  VAR MEAN1-MEAN45;
  FORMAT UCC $AGGFMT.;
  OUTPUT OUT=AVGS2 (DROP= _TYPE_ _FREQ_  RENAME=(UCC= LINE)) SUM= ;
  /* SUM UCC MEANS TO CREATE AGGREGATION SCHEME */
RUN;


  /***************************************************************************/
  /* STEP6: CALCULTATE STANDARD ERRORS                                       */
  /* ----------------------------------------------------------------------- */
  /*  CALCULATE STANDARD ERRORS USING REPLICATE FORMULA                      */
  /***************************************************************************/


DATA SE (KEEP = INCLASS LINE MEAN SE);
  SET AVGS2;
  ARRAY RMNS(44) MEAN1-MEAN44;
  ARRAY DIFF(44) DIFF1-DIFF44;
    DO i = 1 TO 44;
      DIFF(i) = (RMNS(i) - MEAN45)**2;
    END;
  MEAN = MEAN45;
  SE = SQRT((1/44)*SUM(OF DIFF(*)));
RUN;


  /***************************************************************************/
  /* STEP7: TABULATE EXPENDITURES                                            */
  /* ----------------------------------------------------------------------- */
  /* 1 ARRANGE DATA INTO TABULAR FORM                                        */
  /* 2 SET OUT INTERVIEW POPULATIONS FOR POPULATION LINE ITEM                */
  /* 3 INSERT POPULATION LINE INTO TABLE                                     */
  /* 4 INSERT ZERO EXPENDITURE LINE ITEMS INTO TABLE FOR COMPLETENESS        */
  /***************************************************************************/


PROC SORT DATA=SE;
  BY LINE INCLASS;

PROC TRANSPOSE DATA=SE OUT=TAB1
  NAME = ESTIMATE PREFIX = INCLASS;
  BY LINE;
  VAR MEAN SE;
  /*ARRANGE DATA INTO TABULAR FORM */
RUN;


PROC TRANSPOSE DATA=POP (KEEP = SOURCE RPOP45) OUT=CUS
  NAME = LINE PREFIX = INCLASS;
  VAR RPOP45;
  WHERE SOURCE = 'I';
  /* SET ASIDE POPULATIONS FROM INTERVIEW */
RUN;


DATA TAB2;
  SET CUS TAB1;
  IF LINE = 'RPOP45' THEN DO;
    LINE = '100001';
	ESTIMATE = 'N';
	END;
  /* INSERT POPULATION LINE ITEM INTO TABLE AND ASSIGN LINE NUMBER */
RUN;

PROC SORT DATA=TAB2; 
	BY LINE;
RUN;


DATA TAB;
  MERGE TAB2 STUBFILE;
  BY LINE;
    IF LINE NE '100001' THEN DO;
	  IF SURVEY = 'S' THEN DELETE;
	END;
	ARRAY CNTRL(10) INCLASS1-INCLASS10;
	  DO i = 1 TO 10;
	    IF CNTRL(i) = . THEN CNTRL(i) = 0;
		IF SUM(OF CNTRL(*)) = 0 THEN ESTIMATE = 'MEAN';
	  END;

	IF GROUP IN ('CUCHARS' 'INCOME') THEN DO;
	  IF LAG(LINE) = LINE THEN DELETE;
	END;
	IF TITLE = "Percent distribution:" THEN DELETE;
  /* MERGE STUBFILE BACK INTO TABLE TO INSERT EXPENDITURE LINES */
  /* THAT HAD ZERO EXPENDITURES FOR THE YEAR     
*/
	IF SURVEY = 'T' THEN DELETE;
RUN;


PROC TABULATE DATA=TAB;
  CLASS LINE / GROUPINTERNAL ORDER=DATA;
  CLASS ESTIMATE;
  VAR INCLASS1-INCLASS10;
  FORMAT LINE $LBLFMT.;

    TABLE (LINE * ESTIMATE), (INCLASS10 INCLASS1 INCLASS2 INCLASS3 INCLASS4 
                              INCLASS5  INCLASS6 INCLASS7 INCLASS8 INCLASS9) 
    *SUM='' / RTS=25;
    LABEL ESTIMATE=ESTIMATE LINE=LINE
		  INCLASS1='LESS THAN $15,000'    INCLASS2='$15,000 TO $29,999' 
          INCLASS3='$30,000 TO $39,999'   INCLASS4='$40,000 TO $49,999'
          INCLASS5='$50,000 TO $69,999'   INCLASS6='$70,000 TO $99,999'
          INCLASS7='$100,000 TO $149,999' INCLASS8='$150,000 TO $199,999'
          INCLASS9='$200,000 AND OVER'    INCLASS10='ALL CONSUMER UNITS';	
	OPTIONS NODATE NOCENTER NONUMBER LS=167 PS=MAX;
	WHERE LINE NE 'OTHER';
    TITLE "INTEGRATED EXPENDITURES FOR &YEAR BY INCOME BEFORE TAXES";
RUN;


In [ ]:
/***************************************************************************/
  /* PROGRAM NAME:  CEX INTEGRATED SURVEYS SAMPLE PROGRAM (SAS)              */
  /* FUNCTION: CREATE A WEIGHTED CALENDAR YEAR AVERAGE EXPENDITURE           */
  /*           FOR A SPECIFIED GROUP OF UCC's                                */
  /*                                                                         */
  /*                                                                         */
  /***************************************************************************/

  /***************************************************************************/
  /* STEP1: SET PARAMETERS                                                   */
  /***************************************************************************/

  /*ENTER DATA YEAR*/
    %LET YEAR = 2018;
  /* ENTER SURVEY SOURCE FOR WHICH YOU WISH TO ESTIMATE FROM (INTERVIEW, DIARY, INTEGRATED)*/
	%LET ESTIMATE = INTEGRATED;
  /* ENTER THE UCCs YOU WISH TO ESTIMATE FOR*/
	%LET UCC = ('470111'); 
  /* ENTER A NAME FOR THE ESTIMATE BEING CREATED*/
	%LET AGG = GASOLINE;
 
	%LET YR1 = %SUBSTR(&YEAR, 3, 2);
	%LET YR2 = %SUBSTR(%EVAL(&YEAR + 1), 3, 2);
  /*ENTER LOCATION OF THE UNZIPPED MICRODATA FILE*/
    %LET DRIVE = C:\2018_CEX;

	LIBNAME I&YR1 "&DRIVE\INTRVW&YR1";
	LIBNAME D&YR1 "&DRIVE\DIARY&YR1";


  /***************************************************************************/
  /* STEP2: READ IN ALL NEEDED DATA                                          */
  /* ----------------------------------------------------------------------- */
  /* 1 READ IN THE INTERVIEW AND DIARY FMLY FILES & CREATE MO_SCOPE VARIABLE */
  /* 2 READ IN THE INTERVIEW MTAB/ITAB AND DIARY EXPN/DTAB FILES             */
  /* 3 MERGE FMLY AND EXPENDITURE FILES TO DERIVE WEIGHTED EXPENDITURES      */
  /***************************************************************************/


DATA FMLY (KEEP= NEWID QINTRVYR QINTRVMO WTREP01-WTREP44 FINLWT21 REPWT1-REPWT45 MO_SCOPE SOURCE);

SET D&YR1..FMLD&YR1.1 (IN = DIARY)
    D&YR1..FMLD&YR1.2 (IN = DIARY)
    D&YR1..FMLD&YR1.3 (IN = DIARY)
    D&YR1..FMLD&YR1.4 (IN = DIARY)

    I&YR1..FMLI&YR1.1x (IN = INTERVIEW)
    I&YR1..FMLI&YR1.2  (IN = INTERVIEW)
    I&YR1..FMLI&YR1.3  (IN = INTERVIEW)
    I&YR1..FMLI&YR1.4  (IN = INTERVIEW)
    I&YR1..FMLI&YR2.1  (IN = INTERVIEW);

	BY NEWID;

	ESTIMATE = "&ESTIMATE.";

	ARRAY REPS_A(45) WTREP01-WTREP44 FINLWT21;
    ARRAY REPS_B(45) REPWT1-REPWT45;

	IF INTERVIEW AND ESTIMATE IN ("INTERVIEW","INTEGRATED") THEN DO;
		SOURCE = 'I';
		IF QINTRVMO IN ('01','02','03') AND QINTRVYR = &YEAR. THEN MO_SCOPE = (QINTRVMO - 1);
    		ELSE IF QINTRVMO IN ('01','02','03') AND QINTRVYR = &YEAR. + 1 THEN MO_SCOPE = (4 - QINTRVMO);
    			ELSE MO_SCOPE = 3;

 		DO i = 1 TO 45;
	  	IF REPS_A(i) > 0 THEN REPS_B(i) = (REPS_A(i) * MO_SCOPE / 12);
		END;	
				OUTPUT;
				END;

	IF DIARY AND ESTIMATE IN ("DIARY","INTEGRATED") THEN DO;
		SOURCE = "D";
		MO_SCOPE = 3;

	DO i = 1 TO 45;
	  	IF REPS_A(i) > 0 THEN REPS_B(i) = (REPS_A(i) * MO_SCOPE / 12);
		END;	

				OUTPUT;
				END;

	
	  IF QINTRVYR  THEN 
        SOURCE = 'I';
	  IF WEEKI THEN 
        SOURCE = 'D';

RUN;

DATA EXPEND (KEEP= NEWID SOURCE UCC COST REF_YR);

  SET D&YR1..EXPD&YR1.1 (IN = DIARY)
      D&YR1..EXPD&YR1.2 (IN = DIARY)
      D&YR1..EXPD&YR1.3 (IN = DIARY)
      D&YR1..EXPD&YR1.4 (IN = DIARY)

      I&YR1..MTBI&YR1.1X (IN = INTERVIEW)
      I&YR1..MTBI&YR1.2  (IN = INTERVIEW)
      I&YR1..MTBI&YR1.3  (IN = INTERVIEW)
      I&YR1..MTBI&YR1.4  (IN = INTERVIEW)
      I&YR1..MTBI&YR2.1  (IN = INTERVIEW);
	
  ESTIMATE = "&ESTIMATE.";
  IF UCC IN &UCC.;       

	IF INTERVIEW AND ESTIMATE IN ("INTERVIEW") THEN DO;
		SOURCE = 'I';
		IF (REF_YR = "&YEAR") THEN
			OUTPUT;
			END;
	ELSE IF INTERVIEW AND ESTIMATE = "INTEGRATED" THEN DO; 
		SOURCE = "I"; 
		IF (REF_YR = "&YEAR");
		IF PUBFLAG = "2" THEN
 			OUTPUT;
			END;

	IF DIARY AND ESTIMATE IN ("DIARY") THEN DO;
		SOURCE = 'D';
		COST = COST * 13;
			OUTPUT;
			END;

	ELSE IF DIARY AND ESTIMATE IN ("INTEGRATED") THEN DO; 
		SOURCE = "D"; 
		COST = COST * 13;
		IF PUB_FLAG = "2" THEN
 			OUTPUT;
			END;
	
RUN;	

PROC SORT DATA=EXPEND;
	BY NEWID;
RUN;

DATA PUBFILE (KEEP= NEWID SOURCE UCC WTREP01-WTREP44 FINLWT21 RCOST1-RCOST45 REPWT1-REPWT45 MO_SCOPE COST);
  MERGE FMLY   (IN= INFAM)
        EXPEND (IN= INEXP);
  BY NEWID;
  IF (INEXP AND INFAM);

  IF (COST = .)  THEN 
     COST = 0;
     ARRAY REPS_A(45) WTREP01-WTREP44 FINLWT21;
     ARRAY REPS_B(45) RCOST1-RCOST45;

     DO i = 1 TO 45;
	   IF REPS_A(i) > 0
         THEN REPS_B(i) = (REPS_A(i) * COST);
	     ELSE REPS_B(i) = 0; 	
	 END; 

RUN;

  /***************************************************************************/
  /* STEP3: CALCULATE POPULATIONS                                            */
  /* ----------------------------------------------------------------------- */
  /*  SUM ALL 45 WEIGHT VARIABLES TO DERIVE REPLICATE POPULATIONS            */
  /*  FORMATS FOR CORRECT COLUMN CLASSIFICATIONS                             */
  /***************************************************************************/

PROC SUMMARY NWAY DATA=FMLY SUMSIZE=MAX;
  CLASS SOURCE / MLF;
  VAR REPWT1-REPWT45;
  OUTPUT OUT = POP (DROP = _TYPE_ _FREQ_) SUM = RPOP1-RPOP45;
RUN;

 

  /***************************************************************************/
  /* STEP4: CALCULATE WEIGHTED AGGREGATE EXPENDITURES                        */
  /* ----------------------------------------------------------------------- */
  /*  SUM THE 45 REPLICATE WEIGHTED EXPENDITURES TO DERIVE AGGREGATES/UCC    */
  /*  FORMATS FOR CORRECT COLUMN CLASSIFICATIONS                             */
  /***************************************************************************/


PROC SUMMARY NWAY DATA=PUBFILE SUMSIZE=MAX COMPLETETYPES;
  CLASS SOURCE UCC / MLF;
  VAR RCOST1-RCOST45;
  OUTPUT OUT= AGG (DROP= _TYPE_ _FREQ_) 
  SUM= RCOST1-RCOST45;
RUN;



  /***************************************************************************/
  /* STEP5: CALCULTATE MEAN EXPENDITURES                                     */
  /* ----------------------------------------------------------------------- */
  /* 1 READ IN POPULATIONS AND LOAD INTO MEMORY USING A 3 DIMENSIONAL ARRAY  */
  /*   POPULATIONS ARE ASSOCIATED BY INCLASS, SOURCE(t), AND REPLICATE(j)    */
  /* 2 READ IN AGGREGATE EXPENDITURES FROM AGG DATASET                       */
  /* 3 CALCULATE MEANS BY DIVIDING AGGREGATES BY CORRECT SOURCE POPULATIONS  */
  /*   EXPENDITURES SOURCED FROM DIARY ARE CALULATED USING DIARY POPULATIONS */
  /*   WHILE INTRVIEW EXPENDITURES USE INTERVIEW POPULATIONS                 */
  /* 4 SUM EXPENDITURE MEANS PER UCC INTO CORRECT LINE ITEM AGGREGATIONS     */
  /***************************************************************************/


DATA AVGS1 ;
	MERGE 	AGG
			POP; 
		BY SOURCE; 
		
	   		ARRAY REPS_A(45) RCOST1-RCOST45;
			ARRAY REPS_B(45) RPOP1-RPOP45;
			ARRAY REPS_C(45) MEAN1-MEAN45;
     		DO i = 1 TO 45;
	   	IF REPS_A(i) > 0
         THEN REPS_C(i) = (REPS_A(i) / REPS_B(i));
	     ELSE REPS_C(i) = .; 
		END;	
		RUN;


PROC SUMMARY DATA=AVGS1 NWAY COMPLETETYPES;
  CLASS UCC / MLF;
  VAR MEAN1-MEAN45;
  OUTPUT OUT=AVGS2 (DROP= _TYPE_ _FREQ_) SUM= ;
  /* SUM UCC MEANS TO CREATE AGGREGATION SCHEME */
RUN;

  /***************************************************************************/
  /* STEP6: CALCULTATE STANDARD ERRORS                                       */
  /* ----------------------------------------------------------------------- */
  /*  CALCULATE STANDARD ERRORS USING REPLICATE FORMULA                      */
  /***************************************************************************/

PROC SUMMARY DATA=AVGS1 NWAY COMPLETETYPES;
  VAR MEAN1-MEAN45;
  OUTPUT OUT=AVGS_SUM (DROP= _TYPE_ _FREQ_) SUM= ;
  /* SUM UCC MEANS TO CREATE AGGREGATION SCHEME */
RUN;

DATA &ESTIMATE. (KEEP = UCC &ESTIMATE._MEAN&YR1. &ESTIMATE._SE&YR1. &ESTIMATE._CV&YR1.);
RETAIN UCC &ESTIMATE._MEAN&YR1. &ESTIMATE._SE&YR1. &ESTIMATE._CV&YR1.;
FORMAT 	&ESTIMATE._MEAN&YR1. DOLLAR12.2 
		&ESTIMATE._SE&YR1.   8.2
		&ESTIMATE._CV&YR1.   8.2;
LENGTH	UCC $99.;
SET AVGS2 AVGS_SUM;
  ARRAY RMNS(44) MEAN1-MEAN44;
  ARRAY DIFF(44) DIFF1-DIFF44;
    DO i = 1 TO 44;
      DIFF(i) = (RMNS(i) - MEAN45)**2;
    END;
  IF UCC = " " THEN UCC = "&AGG.";
  &ESTIMATE._MEAN&YR1. = MEAN45;
  &ESTIMATE._SE&YR1. = SQRT((1/44)*SUM(OF DIFF(*)));
  &ESTIMATE._CV&YR1. = (&ESTIMATE._SE&YR1. / &ESTIMATE._MEAN&YR1.) * 100; 
  RUN;



In [ ]:
################################################################################
#  PROGRAM NAME:  R SAMPLE CODE FOR COMPUTING A SINGLE CALENDAR YEAR WEIGHTED  #
#                 EXPENDITURE FROM THE PUBLIC USE MICRODATA USING A UCC        #
#                                                                              #
#  WRITTEN BY:    Arcenis Rojas - 29 AUGUST 2017                               #
#  VERSION:       R Version 3.3.3                                              #
#                                                                              #
#  This program outlines the procedures for computing calendar-year, weighted  #
#  expenditure means using the Consumer Expenditure (CE) Interview Survey      #
#  Public Use Microdata (PUMD).                                                #
################################################################################

library(data.table)
library(dplyr)
library(stringr)
library(tidyr)
library(magrittr)
library(readr)

rm(list = ls())

year <- 2016
ucc_exp <- "450110"

# Store the directory containing the data files. It is assumed that the FMLI,
# MTBI, and ITBI files are all in the same folder
data_dir <- "c:/ajr_projects/pumd/2016"


################################################################################
#                                                                              #
#          Read in and stack the monthly expenditure and income files          #
#                                                                              #
################################################################################

# Read in and stack the MTBI files
mtbi <- lapply(
    dir(data_dir, pattern = "^mtbi.*[.]csv$",full.names = TRUE),
    fread,
    select = c("NEWID", "COST", "UCC", "REF_YR"),
    na.strings = c("", ".", "NA")
) %>% bind_rows() %>%

    # Change the column names to lower case
    setnames(old = names(.), new = tolower(names(.))) %>%

    # Filter for expenditures made in the given year and UCC's used for
    # publication
    filter(ref_yr %in% year, ucc %in% ucc_exp) %>%

    # Change "newid" to a character variable
    mutate(newid = as.character(newid)) %>%

    # Remove unnecessary columns
    select(-ref_yr) %>%

    # Group the data by newid and UCC
    group_by(newid) %>%

    # Get the sum of expenditures on each UCC for each newid
    summarise(cost = sum(cost))


################################################################################
#                                                                              #
#                       Read in and stack the FMLI files                       #
#                                                                              #
################################################################################

# Read in and stack the fmli files
fmli <- lapply(
    dir(data_dir, pattern = "^fmli.*[.]csv$", full.names = TRUE),
    fread,
    select = c("NEWID", "FINLWT21", "QINTRVMO", "QINTRVYR"),
    na.strings = c("", ".", "NA")
) %>% bind_rows() %>%
    setnames(old = names(.), new = tolower(names(.))) %>%
    mutate(
        newid = as.character(newid),
        qintrvmo = as.numeric(qintrvmo),

        # Generate a calendar-year population weight variable
        popwt = ifelse(
            qintrvmo %in% 1:3 & qintrvyr %in% year,
            (qintrvmo - 1) / 3 * finlwt21 / 4,
            ifelse(
                qintrvyr %in% (year + 1),
                (4 - qintrvmo) / 3 *finlwt21 / 4,
                finlwt21 / 4
            )
        )
    ) %>%
    select(-c(qintrvyr, qintrvmo))


mean_exp <- left_join(fmli, mtbi, by = "newid") %>%
    mutate(cost = replace(cost, is.na(cost), 0)) %>%
    summarise(mean_exp = sum(cost * finlwt21) / sum(popwt)) %>%
    unlist